### 1. 각 군집의 centroid 부터 거리 계산 (주성분 활용)

In [122]:
import pandas as pd
import numpy as np

pca_result = pd.read_csv('./data/minmax_scaling_PCA_label.csv', index_col=0)

# kmeans centroid numpy array 불러오기
centroids = np.load('./data/kmeans_centroid.npy')

In [123]:
# 클러스터 레이블 별로 데이터프레임 나누기
pca_cluster_0 = pca_result[pca_result['cluster'] == 0]
pca_cluster_1 = pca_result[pca_result['cluster'] == 1]
pca_cluster_2 = pca_result[pca_result['cluster'] == 2]
pca_cluster_3 = pca_result[pca_result['cluster'] == 3]

In [124]:
# 거리 계산 함수 생성
def calculate_distance(a, b):
    distance = 0
    for i in range(len(a)):
        distance += (a[i] - b[i]) ** 2
    return distance ** 0.5

In [101]:
# 오류 메시지 비활성화
import pandas as pd
pd.options.mode.chained_assignment = None

In [125]:
# 각 cluster centroid로부터 떨어진 거리 계산 -> 'distance' 컬럼
pca_clusters = [pca_cluster_0, pca_cluster_1, pca_cluster_2, pca_cluster_3]

for i, pca_cluster in enumerate(pca_clusters):
    tmp = pca_cluster.drop('cluster', axis=1)
    distances = [calculate_distance(centroids[i], point) for point in tmp.values]
    pca_cluster.loc[:,'distance'] = distances

클러스터별 최단 거리 5위 뽑아보기

In [127]:
pd.concat([
    pca_cluster_0.sort_values('distance').head(),
    pca_cluster_1.sort_values('distance').head(),
    pca_cluster_2.sort_values('distance').head(),
    pca_cluster_3.sort_values('distance').head(),
])

,0,1,2,3,4,5,cluster,distance
행정동명,,,,,,,,
송파2동,0.079815,-0.244630,0.027826,0.015311,-0.039013,-0.010445,0,0.143294
남가좌1동,0.088593,-0.312289,0.017162,-0.167120,0.047328,0.041055,0,0.158830
하계2동,0.164587,-0.306063,-0.004596,-0.099306,-0.003715,-0.035986,0,0.163167
방이1동,-0.056980,-0.250338,0.119408,-0.080721,-0.087399,0.055004,0,0.168104
방배3동,0.100324,-0.300655,0.057266,0.061770,0.054492,0.053401,0,0.173730
면목4동,-0.075829,-0.078424,-0.163650,0.052472,0.042259,0.000171,1,0.089658
암사1동,-0.065351,-0.064685,-0.098996,-0.018784,0.066846,0.051345,1,0.093978
홍제3동,-0.044947,-0.102270,-0.078139,0.104911,0.052648,-0.021009,1,0.110024
화곡2동,-0.077616,-0.105319,-0.091817,-0.046530,-0.033173,0.018272,1,0.112856


### 2. stanine을 이용해서 클러스터별 등급 매기기

In [134]:
import pandas as pd
import numpy as np

pca_result = pd.read_csv('./data/minmax_scaling_PCA_label.csv', index_col=0)
scaled_df = pd.read_csv('./data/minmax_scaling.csv', index_col=0)
original_df = pd.read_csv('./data/final_data_after_winsorization_with_selected_features.csv', index_col=0)

## 최종 입지 선정